In [2]:
import torch
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import defaultdict

sys.path.append(os.path.abspath('../src'))
from embeddings import PitchEmbedding

In [3]:
# Check and use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Using device: cuda
GPU: NVIDIA GeForce MX250
Memory: 2.15 GB


## Random Sample of rows to test PitchEmbedding function

In [3]:
# Memory-efficient random sampling from large file
import random
import os

# Set random seed
random.seed(42)

# Parameters
sample_size = 500000
input_file = '../data/processed/pitcher_final_21-24.csv'
output_file = '../data/processed/pitcher_test_sample.csv'

# Count total lines first (optional but helps with progress reporting)
with open(input_file, 'r') as f:
    total_lines = sum(1 for _ in f)

print(f"Total lines in file: {total_lines}")

# Calculate sampling probability
sampling_probability = sample_size / (total_lines - 1)  # Exclude header

# Perform reservoir sampling
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    # Copy header
    header = next(infile)
    outfile.write(header)
    
    # Process remaining lines with sampling
    lines_sampled = 0
    for i, line in enumerate(infile):
        if random.random() < sampling_probability:
            outfile.write(line)
            lines_sampled += 1
            
        # Report progress every million lines
        if (i+1) % 1000000 == 0:
            print(f"Processed {i+1:,} lines...")
            
    print(f"Sampled {lines_sampled} lines from {total_lines:,} total")

# Load the much smaller sample file
df = pd.read_csv(output_file)
print(f"Successfully loaded {len(df)} rows")

Total lines in file: 3073583
Processed 1,000,000 lines...
Processed 2,000,000 lines...
Processed 3,000,000 lines...
Sampled 500788 lines from 3,073,583 total


C:\Users\ryanr\AppData\Local\Temp\ipykernel_15640\295004641.py:42: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_file)


Successfully loaded 500788 rows


In [5]:
# Read your CSV file
#df = pd.read_csv('../data/processed/pitcher_final_21-24.csv')

# Convert problematic columns to appropriate types
# Convert 'zone' column to numeric, handling any string values
df['zone'] = pd.to_numeric(df['zone'], errors='coerce')

# Handle other potential problematic columns
for col in df.select_dtypes(include=['object']):
	# Try to convert string representations of numbers to actual numeric values
	try:
		df[col] = pd.to_numeric(df[col], errors='coerce')
	except:
		pass

# Save as Parquet (install pyarrow if needed: pip install pyarrow)
df.to_parquet('../data/processed/pitcher_final_21-24.parquet', engine='pyarrow')

print("Conversion complete!")

Conversion complete!


In [7]:
# 1. LOAD PROCESSED DATA
print("Loading processed data...")
# Use your engineered dataset
#df = pd.read_csv('../data/processed/pitcher_final_21-24.parquet')
df = pd.read_parquet('../data/processed/pitcher_final_21-24.parquet')

# 2. DEFINE CONTINUOUS & CATEGORICAL FEATURES
continuous_features = [
    'release_speed', 'release_pos_x', 'release_pos_z', 
    'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y',
    'vx0', 'vy0', 'vz0', 'ax', 'ay', 
    'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 
    'launch_speed', 'launch_angle', 'effective_speed', 
    'release_spin_rate', 'release_extension', 
    'release_pos_y', 'estimated_ba_using_speedangle',
    'estimated_woba_using_speedangle', 'woba_value',
    'woba_denom', 'babip_value', 'iso_value', 'spin_axis',
    'delta_home_win_exp', 'delta_run_exp', 'bat_speed' ,
    'swing_length', 'estimated_slg_using_speedangle',
    'delta_pitcher_run_exp', 'hyper_speed',
    'bat_win_exp',
    'pitcher_days_since_prev_game', 
    'batter_days_since_prev_game',
    'pitcher_days_until_next_game',
    'batter_days_until_next_game',
    'api_break_z_with_gravity',
    'api_break_x_arm',
    'api_break_x_batter_in',
    'arm_angle', 'home_win_exp',

    'at_bat_number', 'bat_score', 'fld_score', 'age_pit', 'age_bat'
]

categorical_features = [
    'pitch_type', 'batter', 'pitcher', 'events', 
    #'description', 
    'zone', 
    'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 
    'result_type',
    'hit_location', 'bb_type', 'balls', 'strikes', 'game_year',
    'outs_when_up', 'inning', 'inning_topbot', 'game_pk',
    'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
    'fielder_7', 'fielder_8', 'fielder_9', 
    #'at_bat_number', 
    'pitch_number',
    'launch_speed_angle', 
    #'home_score', 'away_score', 
    #'bat_score',
    #'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score', 
    'if_fielding_alignment', 'of_fielding_alignment',
    #'age_pit_legacy', 'age_bat_legacy', 
    #'age_pit', 'age_bat',
    'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
    #'pitch_name',
    'on_1b', 'on_2b', 'on_3b',

    'at_bat_id', 'month','score_diff', 'risp', 'clutch', 'blowout', 'post_allstar_break'
]


# 3. NORMALIZE CONTINUOUS FEATURES
print("Normalizing continuous features...")
scaler = StandardScaler()
df_cont = df[continuous_features].copy()
# Handle any remaining NaNs
df_cont = df_cont.fillna(0)
df_cont_scaled = scaler.fit_transform(df_cont)

# 4. CREATE CATEGORY MAPPINGS & CONVERT TO INDICES
print("Converting categorical features to indices...")
category_maps = {}
cat_data = {}

# Define custom embedding dimensions for important features
# Format: 'feature_name': desired_embedding_dimension
custom_embedding_dims = {
    'pitch_type': 16, 
    'batter': 16, 
    'pitcher': 16, 
    'events': 12, 
    #'description', 
    'zone': 6, 
    'game_type': 4, 
    'stand': 4, 
    'p_throws': 4, 
    'home_team': 8, 
    'away_team': 8, 
    'result_type': 4,
    'hit_location': 6, 
    'bb_type': 8, 
    'balls': 4, 
    'strikes': 4, 
    'game_year': 4,
    'outs_when_up': 8, 
    'inning': 16, 
    'inning_topbot': 2, 
    'game_pk': 16,
    #'fielder_2':, 
    #'fielder_3':, 
    #'fielder_4':, 
    #'fielder_5':, 
    #'fielder_6':,
    #'fielder_7':, 
    #'fielder_8':, 
    #'fielder_9':, 
    ####'at_bat_number': 12, 
    'pitch_number': 12,
    'launch_speed_angle': 12, 
    #'home_score', 
    #'away_score', 
    ####'bat_score': 16,
    ####'fld_score': 16, 
    #'post_away_score', 
    #'post_home_score', 
    #'post_bat_score',
    #'post_fld_score', 
    'if_fielding_alignment': 4, 
    'of_fielding_alignment': 4,
    #'age_pit_legacy', 
    #'age_bat_legacy', 
    ####'age_pit': 8, 
    ####'age_bat': 8,
    'n_thruorder_pitcher': 12, 
    'n_priorpa_thisgame_player_at_bat': 12,
    #'pitch_name',
    
    'on_1b': 4,
    'on_2b': 4,
    'on_3b': 4,

    'at_bat_id': 16, 
    'month': 6,
    'score_diff': 8, 
    'risp': 12, 
    'clutch': 12,
    'blowout': 4, 
    'post_allstar_break': 4
}

for feature in categorical_features:
    # Create a mapping from category values to indices
    unique_values = df[feature].dropna().unique()
    category_maps[feature] = {val: idx for idx, val in enumerate(unique_values)}
    
    # Use custom dimension if specified, otherwise use 2
    if feature in custom_embedding_dims:
        embedding_dim = custom_embedding_dims[feature]
    else:
        # Fallback to heuristic
        #embedding_dim = min(50, max(4, int(np.sqrt(len(unique_values)) * 2)))
        embedding_dim = 2
    
    # Store category counts and embedding dimensions
    cat_data[feature] = {
        'num_categories': len(unique_values) + 1,  # +1 for unknown/padding
        'embedding_dim': embedding_dim
    }
    
    # Print some info
    print(f"{feature}: {len(unique_values)} categories, {cat_data[feature]['embedding_dim']} dimensions")



# 5. CONVERT TO PYTORCH TENSORS
print("Creating tensors...")
# Create a tensor for all continuous features
continuous_tensor = torch.tensor(df_cont_scaled, dtype=torch.float32).to(device)

# Create tensors for categorical features
categorical_tensors = {}
for feature in categorical_features:
    # Map values to indices, using 0 for NaN
    indices = [category_maps[feature].get(val, 0) for val in df[feature].fillna('unknown')]
    categorical_tensors[feature] = torch.tensor(indices, dtype=torch.long).to(device)

# 6. DEFINE MODEL PARAMETERS
continuous_dim = len(continuous_features)
#output_dim = 256  # Final embedding dimension
output_dim = 128  # Final embedding dimension (smaller for prototype)

# 7. CREATE EMBEDDING MODEL
print("Creating embedding model...")
embedding_model = PitchEmbedding(continuous_dim, cat_data, output_dim).to(device)

# 8. GENERATE EMBEDDINGS
print("Generating embeddings...")
with torch.no_grad():  # No need for gradients during inference
    pitch_embeddings = embedding_model(continuous_tensor, categorical_tensors)

print(f"Created embeddings with shape: {pitch_embeddings.shape}")

# 9. ORGANIZE INTO AT-BAT SEQUENCES
print("Organizing into at-bat sequences...")
# Group by at_bat_id
unique_at_bats = df['at_bat_id'].unique()
print(f"Total at-bats: {len(unique_at_bats)}")

# Store atbat sequences and their lengths
atbat_sequences = []
atbat_lengths = []
max_seq_len = 0

# Process a subset for demonstration (adjust as needed)
#sample_at_bats = unique_at_bats[:1000]     #.. (try with only 1000 at-bats for testing)
# sample_at_bats = unique_at_bats[:5000]    #.. (try it with more data for more testing)
sample_at_bats = unique_at_bats           #.. (use all data for production)


for at_bat_id in sample_at_bats:
    # Get indices for this at-bat
    indices = df[df['at_bat_id'] == at_bat_id].index
    
    # Get embeddings for these pitches
    at_bat_embeddings = pitch_embeddings[indices]
    
    # Update max sequence length
    max_seq_len = max(max_seq_len, len(indices))
    
    # Store sequence and length
    atbat_sequences.append(at_bat_embeddings)
    atbat_lengths.append(len(indices))

print(f"Processed {len(atbat_sequences)} at-bat sequences")
print(f"Average pitches per at-bat: {np.mean(atbat_lengths):.2f}")
print(f"Maximum sequence length: {max_seq_len}")

# 10. PAD SEQUENCES FOR BATCH PROCESSING
print("Padding sequences...")
padded_sequences = []
attention_mask = []

for seq in atbat_sequences:
    seq_len = seq.shape[0]
    
    # Create padded sequence
    padded = torch.zeros(max_seq_len, output_dim, device=device)
    padded[:seq_len] = seq
    
    # Create mask (1 for real tokens, 0 for padding)
    mask = torch.zeros(max_seq_len, device=device)
    mask[:seq_len] = 1
    
    padded_sequences.append(padded)
    attention_mask.append(mask)

# Stack into tensors
padded_tensor = torch.stack(padded_sequences).to(device)
attention_mask = torch.stack(attention_mask).bool().to(device)

print(f"Final tensor shape: {padded_tensor.shape}")
print(f"Attention mask shape: {attention_mask.shape}")

# 11. SAVE FOR LATER USE
torch.save({
    'embeddings': padded_tensor.cpu(),
    'attention_mask': attention_mask.cpu(),
    'scaler': scaler,
    'category_maps': category_maps
}, '../data/embeddings/pitch_embeddings_test_parquet.pt')

print("Embeddings saved and ready for transformer training!")

Loading processed data...
Normalizing continuous features...
Converting categorical features to indices...
pitch_type: 0 categories, 16 dimensions
batter: 2481 categories, 16 dimensions
pitcher: 2218 categories, 16 dimensions
events: 0 categories, 12 dimensions
zone: 13 categories, 6 dimensions
game_type: 0 categories, 4 dimensions
stand: 0 categories, 4 dimensions
p_throws: 0 categories, 4 dimensions
home_team: 0 categories, 8 dimensions
away_team: 0 categories, 8 dimensions
result_type: 0 categories, 4 dimensions
hit_location: 9 categories, 6 dimensions
bb_type: 0 categories, 8 dimensions
balls: 4 categories, 4 dimensions
strikes: 3 categories, 4 dimensions
game_year: 4 categories, 4 dimensions
outs_when_up: 3 categories, 8 dimensions
inning: 18 categories, 16 dimensions
inning_topbot: 0 categories, 2 dimensions
game_pk: 10705 categories, 16 dimensions
fielder_2: 363 categories, 2 dimensions
fielder_3: 656 categories, 2 dimensions
fielder_4: 714 categories, 2 dimensions
fielder_5: 71

In [8]:
torch.cuda.empty_cache()

# 2025

In [4]:
# Read your CSV file
df = pd.read_csv('../data/processed/pitcher_final_2025.csv')

# Convert problematic columns to appropriate types
# Convert 'zone' column to numeric, handling any string values
df['zone'] = pd.to_numeric(df['zone'], errors='coerce')

# Handle other potential problematic columns
for col in df.select_dtypes(include=['object']):
	# Try to convert string representations of numbers to actual numeric values
	try:
		df[col] = pd.to_numeric(df[col], errors='coerce')
	except:
		pass

# Save as Parquet (install pyarrow if needed: pip install pyarrow)
df.to_parquet('../data/processed/pitcher_final_2025.parquet', engine='pyarrow')

print("Conversion complete!")

C:\Users\ryanr\AppData\Local\Temp\ipykernel_19916\293898324.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/pitcher_final_2025.csv')


Conversion complete!


In [5]:
# 1. LOAD PROCESSED DATA
print("Loading processed data...")
# Use your engineered dataset
#df = pd.read_csv('../data/processed/pitcher_final_21-24.parquet')
df = pd.read_parquet('../data/processed/pitcher_final_2025.parquet')

# 2. DEFINE CONTINUOUS & CATEGORICAL FEATURES
continuous_features = [
    'release_speed', 'release_pos_x', 'release_pos_z', 
    'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y',
    'vx0', 'vy0', 'vz0', 'ax', 'ay', 
    'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 
    'launch_speed', 'launch_angle', 'effective_speed', 
    'release_spin_rate', 'release_extension', 
    'release_pos_y', 'estimated_ba_using_speedangle',
    'estimated_woba_using_speedangle', 'woba_value',
    'woba_denom', 'babip_value', 'iso_value', 'spin_axis',
    'delta_home_win_exp', 'delta_run_exp', 'bat_speed' ,
    'swing_length', 'estimated_slg_using_speedangle',
    'delta_pitcher_run_exp', 'hyper_speed',
    'bat_win_exp',
    'pitcher_days_since_prev_game', 
    'batter_days_since_prev_game',
    'pitcher_days_until_next_game',
    'batter_days_until_next_game',
    'api_break_z_with_gravity',
    'api_break_x_arm',
    'api_break_x_batter_in',
    'arm_angle', 'home_win_exp',

    'at_bat_number', 'bat_score', 'fld_score', 'age_pit', 'age_bat'
]

categorical_features = [
    'pitch_type', 'batter', 'pitcher', 'events', 
    #'description', 
    'zone', 
    'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 
    'result_type',
    'hit_location', 'bb_type', 'balls', 'strikes', 'game_year',
    'outs_when_up', 'inning', 'inning_topbot', 'game_pk',
    'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
    'fielder_7', 'fielder_8', 'fielder_9', 
    #'at_bat_number', 
    'pitch_number',
    'launch_speed_angle', 
    #'home_score', 'away_score', 
    #'bat_score',
    #'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score', 
    'if_fielding_alignment', 'of_fielding_alignment',
    #'age_pit_legacy', 'age_bat_legacy', 
    #'age_pit', 'age_bat',
    'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
    #'pitch_name',
    'on_1b', 'on_2b', 'on_3b',

    'at_bat_id', 'month','score_diff', 'risp', 'clutch', 'blowout', 'post_allstar_break'
]


# 3. NORMALIZE CONTINUOUS FEATURES
print("Normalizing continuous features...")
scaler = StandardScaler()
df_cont = df[continuous_features].copy()
# Handle any remaining NaNs
df_cont = df_cont.fillna(0)
df_cont_scaled = scaler.fit_transform(df_cont)

# 4. CREATE CATEGORY MAPPINGS & CONVERT TO INDICES
print("Converting categorical features to indices...")
category_maps = {}
cat_data = {}

# Define custom embedding dimensions for important features
# Format: 'feature_name': desired_embedding_dimension
custom_embedding_dims = {
    'pitch_type': 16, 
    'batter': 16, 
    'pitcher': 16, 
    'events': 12, 
    #'description', 
    'zone': 6, 
    'game_type': 4, 
    'stand': 4, 
    'p_throws': 4, 
    'home_team': 8, 
    'away_team': 8, 
    'result_type': 4,
    'hit_location': 6, 
    'bb_type': 8, 
    'balls': 4, 
    'strikes': 4, 
    'game_year': 4,
    'outs_when_up': 8, 
    'inning': 16, 
    'inning_topbot': 2, 
    'game_pk': 16,
    #'fielder_2':, 
    #'fielder_3':, 
    #'fielder_4':, 
    #'fielder_5':, 
    #'fielder_6':,
    #'fielder_7':, 
    #'fielder_8':, 
    #'fielder_9':, 
    ####'at_bat_number': 12, 
    'pitch_number': 12,
    'launch_speed_angle': 12, 
    #'home_score', 
    #'away_score', 
    ####'bat_score': 16,
    ####'fld_score': 16, 
    #'post_away_score', 
    #'post_home_score', 
    #'post_bat_score',
    #'post_fld_score', 
    'if_fielding_alignment': 4, 
    'of_fielding_alignment': 4,
    #'age_pit_legacy', 
    #'age_bat_legacy', 
    ####'age_pit': 8, 
    ####'age_bat': 8,
    'n_thruorder_pitcher': 12, 
    'n_priorpa_thisgame_player_at_bat': 12,
    #'pitch_name',
    
    'on_1b': 4,
    'on_2b': 4,
    'on_3b': 4,

    'at_bat_id': 16, 
    'month': 6,
    'score_diff': 8, 
    'risp': 12, 
    'clutch': 12,
    'blowout': 4, 
    'post_allstar_break': 4
}

for feature in categorical_features:
    # Create a mapping from category values to indices
    unique_values = df[feature].dropna().unique()
    category_maps[feature] = {val: idx for idx, val in enumerate(unique_values)}
    
    # Use custom dimension if specified, otherwise use 2
    if feature in custom_embedding_dims:
        embedding_dim = custom_embedding_dims[feature]
    else:
        # Fallback to heuristic
        #embedding_dim = min(50, max(4, int(np.sqrt(len(unique_values)) * 2)))
        embedding_dim = 2
    
    # Store category counts and embedding dimensions
    cat_data[feature] = {
        'num_categories': len(unique_values) + 1,  # +1 for unknown/padding
        'embedding_dim': embedding_dim
    }
    
    # Print some info
    print(f"{feature}: {len(unique_values)} categories, {cat_data[feature]['embedding_dim']} dimensions")



# 5. CONVERT TO PYTORCH TENSORS
print("Creating tensors...")
# Create a tensor for all continuous features
continuous_tensor = torch.tensor(df_cont_scaled, dtype=torch.float32).to(device)

# Create tensors for categorical features
categorical_tensors = {}
for feature in categorical_features:
    # Map values to indices, using 0 for NaN
    indices = [category_maps[feature].get(val, 0) for val in df[feature].fillna('unknown')]
    categorical_tensors[feature] = torch.tensor(indices, dtype=torch.long).to(device)

# 6. DEFINE MODEL PARAMETERS
continuous_dim = len(continuous_features)
#output_dim = 256  # Final embedding dimension
output_dim = 128  # Final embedding dimension (smaller for prototype)

# 7. CREATE EMBEDDING MODEL
print("Creating embedding model...")
embedding_model = PitchEmbedding(continuous_dim, cat_data, output_dim).to(device)

# 8. GENERATE EMBEDDINGS
print("Generating embeddings...")
with torch.no_grad():  # No need for gradients during inference
    pitch_embeddings = embedding_model(continuous_tensor, categorical_tensors)

print(f"Created embeddings with shape: {pitch_embeddings.shape}")

# 9. ORGANIZE INTO AT-BAT SEQUENCES
print("Organizing into at-bat sequences...")
# Group by at_bat_id
unique_at_bats = df['at_bat_id'].unique()
print(f"Total at-bats: {len(unique_at_bats)}")

# Store atbat sequences and their lengths
atbat_sequences = []
atbat_lengths = []
max_seq_len = 0

# Process a subset for demonstration (adjust as needed)
#sample_at_bats = unique_at_bats[:1000]     #.. (try with only 1000 at-bats for testing)
# sample_at_bats = unique_at_bats[:5000]    #.. (try it with more data for more testing)
sample_at_bats = unique_at_bats           #.. (use all data for production)


for at_bat_id in sample_at_bats:
    # Get indices for this at-bat
    indices = df[df['at_bat_id'] == at_bat_id].index
    
    # Get embeddings for these pitches
    at_bat_embeddings = pitch_embeddings[indices]
    
    # Update max sequence length
    max_seq_len = max(max_seq_len, len(indices))
    
    # Store sequence and length
    atbat_sequences.append(at_bat_embeddings)
    atbat_lengths.append(len(indices))

print(f"Processed {len(atbat_sequences)} at-bat sequences")
print(f"Average pitches per at-bat: {np.mean(atbat_lengths):.2f}")
print(f"Maximum sequence length: {max_seq_len}")

# 10. PAD SEQUENCES FOR BATCH PROCESSING
print("Padding sequences...")
padded_sequences = []
attention_mask = []

for seq in atbat_sequences:
    seq_len = seq.shape[0]
    
    # Create padded sequence
    padded = torch.zeros(max_seq_len, output_dim, device=device)
    padded[:seq_len] = seq
    
    # Create mask (1 for real tokens, 0 for padding)
    mask = torch.zeros(max_seq_len, device=device)
    mask[:seq_len] = 1
    
    padded_sequences.append(padded)
    attention_mask.append(mask)

# Stack into tensors
padded_tensor = torch.stack(padded_sequences).to(device)
attention_mask = torch.stack(attention_mask).bool().to(device)

print(f"Final tensor shape: {padded_tensor.shape}")
print(f"Attention mask shape: {attention_mask.shape}")

# 11. SAVE FOR LATER USE
torch.save({
    'embeddings': padded_tensor.cpu(),
    'attention_mask': attention_mask.cpu(),
    'scaler': scaler,
    'category_maps': category_maps
}, '../data/embeddings/pitch_embeddings_2025_parquet.pt')

print("Embeddings saved and ready for transformer training!")

Loading processed data...
Normalizing continuous features...
Converting categorical features to indices...
pitch_type: 0 categories, 16 dimensions
batter: 1355 categories, 16 dimensions
pitcher: 900 categories, 16 dimensions
events: 0 categories, 12 dimensions
zone: 13 categories, 6 dimensions
game_type: 0 categories, 4 dimensions
stand: 0 categories, 4 dimensions
p_throws: 0 categories, 4 dimensions
home_team: 0 categories, 8 dimensions
away_team: 0 categories, 8 dimensions
result_type: 0 categories, 4 dimensions
hit_location: 9 categories, 6 dimensions
bb_type: 0 categories, 8 dimensions
balls: 5 categories, 4 dimensions
strikes: 4 categories, 4 dimensions
game_year: 1 categories, 4 dimensions
outs_when_up: 3 categories, 8 dimensions
inning: 12 categories, 16 dimensions
inning_topbot: 0 categories, 2 dimensions
game_pk: 216 categories, 16 dimensions
fielder_2: 194 categories, 2 dimensions
fielder_3: 234 categories, 2 dimensions
fielder_4: 277 categories, 2 dimensions
fielder_5: 247 c